# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import json
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rikitea,-23.1203,-134.9692,26.85,81,49,6.79,PF,2023-04-03
1,1,sidi ali,36.0998,0.4206,12.28,50,0,2.26,DZ,2023-04-03
2,2,constitucion,-35.3333,-72.4167,13.71,62,23,3.95,CL,2023-04-03
3,3,narsaq,60.9167,-46.0500,7.12,73,100,2.80,GL,2023-04-03
4,4,cidreira,-30.1811,-50.2056,20.18,76,7,5.80,BR,2023-04-03


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_humidity_map = city_data_df.hvplot.points(
"Lng",
"Lat",
geo = True,
tiles = "OSM",
frame_width = 700,
frame_height = 600,
size = "Humidity",
scale = 1,
color = "City",
hover_cols=[])

# Display the map
city_humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# I love the Tropic of Capricorn at latitude -23.26 degrees.
# The fact that there are 3 completely different deserts Atacama Chile South America at -23.86 Lat
# The Kalahari in Africa Botswana, South Africa and Namibia at -25.59 Lat
# The Simpson desert in Australia at -25 degrees lat
# And perversely Rio De Janeiro with a climate against form at -22.9 Lat
# I am choosing a narrow band across the desert latitude from -22 Lat to -26
# I am excluding South Africa as travel smartraveller.gov.au website advises "exercise extreme caution". 
# Namibia and Botswana have Australian government advise to travellers of 
# 'Exercise Normal safety precautions' and so I will include these cities



holiday_df = city_data_df.loc[((city_data_df['Lat'] <= -22) & (city_data_df['Lat'] >= -26)) & (city_data_df['Country'] != 'SA')].reset_index(drop=True).copy()


# Drop any rows with null values
# YOUR CODE HERE
holiday_df.dropna(how='any', inplace=True)

# Display sample data
holiday_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rikitea,-23.1203,-134.9692,26.85,81,49,6.79,PF,2023-04-03
1,13,arraial do cabo,-22.9661,-42.0278,24.32,70,100,3.80,BR,2023-04-03
2,39,castro,-24.7911,-50.0119,12.44,97,72,3.26,BR,2023-04-03
3,52,carnarvon,-24.8667,113.6333,25.98,68,48,5.51,AU,2023-04-03
4,62,yeppoon,-23.1333,150.7333,26.88,77,52,5.71,AU,2023-04-03
5,122,inhambane,-23.8650,35.3833,27.99,65,40,1.95,MZ,2023-04-03
6,176,letlhakeng,-24.0944,25.0298,23.12,49,11,4.03,BW,2023-04-03
7,191,vao,-22.6667,167.4833,25.86,79,26,4.53,NC,2023-04-03
8,229,yulara,-25.2406,130.9889,31.65,24,0,1.98,AU,2023-04-03
9,250,beloha,-25.1667,45.0500,27.61,59,25,4.48,MG,2023-04-03


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = holiday_df.loc[:, ["City", "Country", "Lat","Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None
# Display sample data  
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,rikitea,PF,-23.1203,-134.9692,81,None
1,arraial do cabo,BR,-22.9661,-42.0278,70,None
2,castro,BR,-24.7911,-50.0119,97,None
3,carnarvon,AU,-24.8667,113.6333,68,None
4,yeppoon,AU,-23.1333,150.7333,77,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
#I have used coding example wk6 day 3 exercise 4 for this section
# Set parameters to search for a hotel
limit = 20
radius = 5000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row["Lng"]
    lat = row["Lat"]
  

    # Set up a parameters dictionary for the current city's latitude and longitude
    params = {
        "type": "accommodation.hotel",
        "filter": f"circle:{lon},{lat},{radius}",
        "limit": limit,
        "bias": f"proximity:{lon},{lat}",
        "apiKey": geoapify_key    
    }

    #Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    #print(name_address.url)
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{row['City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
rikitea - nearest hotel: Chez Bianca & Benoit
arraial do cabo - nearest hotel: No hotel found
castro - nearest hotel: Hotel Central Palace
carnarvon - nearest hotel: No hotel found
yeppoon - nearest hotel: Blossom's on Seaspray
inhambane - nearest hotel: Hotel de Inhambane
letlhakeng - nearest hotel: No hotel found
vao - nearest hotel: Nataiwatch
yulara - nearest hotel: Outback Pioneer Hotel & Lodge
beloha - nearest hotel: Hôtel Oliv
villazon - nearest hotel: Hotel Ideal
santa isabel - nearest hotel: No hotel found
tocopilla - nearest hotel: hotel Isidora
emerald - nearest hotel: Emerald Central Hotel
alice springs - nearest hotel: Aurora Alice Springs
paranapanema - nearest hotel: No hotel found
itaguai - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,rikitea,PF,-23.1203,-134.9692,81,Chez Bianca & Benoit
1,arraial do cabo,BR,-22.9661,-42.0278,70,No hotel found
2,castro,BR,-24.7911,-50.0119,97,Hotel Central Palace
3,carnarvon,AU,-24.8667,113.6333,68,No hotel found
4,yeppoon,AU,-23.1333,150.7333,77,Blossom's on Seaspray
5,inhambane,MZ,-23.8650,35.3833,65,Hotel de Inhambane
6,letlhakeng,BW,-24.0944,25.0298,49,No hotel found
7,vao,NC,-22.6667,167.4833,79,Nataiwatch
8,yulara,AU,-25.2406,130.9889,24,Outback Pioneer Hotel & Lodge
9,beloha,MG,-25.1667,45.0500,59,Hôtel Oliv


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display
# Configure the map plot
Tropic_Capricorn_Hotel_Map = hotel_df.hvplot.points(
"Lng",
"Lat",
geo = True,
tiles = "OSM",
frame_width = 700,
frame_height = 300,
#size = "Humidity",
scale = 1,
color = "City",
hover_cols=["Country","Hotel Name"])

# Display the map
Tropic_Capricorn_Hotel_Map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)